In [1]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [14]:
df = pd.read_csv("P-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 1009

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,punts,yds,avg,hasbling,hasawards,allamerican,heisman,drafted
0,2012,bryan anger,California,P,75.63,208,4.86,NaN,NaN,NaN,NaN,NaN,255.0,11094.0,43.5,0.0,0.0,0.0,0.0,Y
1,2013,dylan breeding,Arkansas,P,73.25,213,4.74,11.0,29.0,110.0,NaN,NaN,219.0,9387.0,42.9,0.0,0.0,0.0,0.0,N
2,2013,josh hubner,Arizona State,P,75.63,232,5.06,13.0,NaN,NaN,NaN,NaN,114.0,5013.0,44.0,0.0,0.0,0.0,0.0,N
3,2014,cody mandell,Alabama,P,74.50,216,4.89,NaN,27.0,NaN,NaN,NaN,169.0,7191.0,42.6,0.0,0.0,0.0,0.0,N
4,2014,pat odonnell,Miami (FL),P,76.00,220,4.64,23.0,30.5,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,Y


In [10]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
for index, row in df.iterrows():
    if (df.loc[index,"hasbling"] != 1.0):
        if (df.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                df.loc[index, "punts"] = browser.find_by_css("table#punting tfoot tr.bold")[0].find_by_css("td[data-stat='punt']").text
                df.loc[index, "yds"] = browser.find_by_css("table#punting tfoot tr.bold")[0].find_by_css("td[data-stat='punt_yds']").text
                df.loc[index, "avg"] = browser.find_by_css("table#punting tfoot tr.bold")[0].find_by_css("td[data-stat='punt_yds_per_punt']").text
                            
                if (len(browser.find_by_css("ul#bling")) > 0):
                    df.loc[index,"hasbling"] = 1
                else:
                    df.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"hasawards"] = 1
                else:
                    df.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"hasbling"] = 0
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            df.loc[index, "punts"] = browser.find_by_css("table#punting tfoot tr.bold")[0].find_by_css("td[data-stat='punt']").text
                            df.loc[index, "yds"] = browser.find_by_css("table#punting tfoot tr.bold")[0].find_by_css("td[data-stat='punt_yds']").text
                            df.loc[index, "avg"] = browser.find_by_css("table#punting tfoot tr.bold")[0].find_by_css("td[data-stat='punt_yds_per_punt']").text

                            if (len(browser.find_by_css("ul#bling")) > 0):
                                df.loc[index,"hasbling"] = 1
                            else:
                                df.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"hasawards"] = 1
                            else:
                                df.loc[index,"hasawards"] = 0
                            break
                        else:
                            df.loc[index,"hasbling"] = 0
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"hasawards"] = 0
                            break
                    else:
                        df.loc[index,"hasbling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0

In [16]:
df.to_csv("P-Train.csv", index=False)